# Collection setup and data loading

Learn how to create Weaviate collections and import data with vectorization. We'll create the FinancialArticles collection used in other notebooks.

## Connect to Weaviate

Connect to a Weaviate instance using environment variables.

In [1]:
import weaviate
import os
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

# client = weaviate.connect_to_custom(
#     http_host="<http_host>",
#     http_port="<http_port>",
#     grpc_host="<grpc_host>",
#     grpc_port="<grpc_port>",
# )

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=os.environ["WCD_TEST_URL"],
    auth_credentials=os.environ["WCD_TEST_KEY"]
)

client.is_ready()

True

## Create collection with vectorizer

Create a collection for financial articles with separate embeddings for title and content.

[Docs - Collection configuration](https://weaviate.io/developers/weaviate/manage-data/collections)

In [2]:
from weaviate.classes.config import Configure, Property, DataType

# Delete collection if it exists
if client.collections.exists("FinancialArticles"):
    client.collections.delete("FinancialArticles")

# Create collection with named vectors for title and content
client.collections.create(
    name="FinancialArticles",

    # Configure separate embeddings for title and article content
    vector_config=[
        Configure.Vectors.text2vec_weaviate(
            name="title",
            model="Snowflake/snowflake-arctic-embed-l-v2.0",
            source_properties=["article_title"]
        ),
        Configure.Vectors.text2vec_weaviate(
            name="content",
            model="Snowflake/snowflake-arctic-embed-l-v2.0",
            source_properties=["article"]
        )
    ],

    # Define property schema
    properties=[
        Property(name="article_title", data_type=DataType.TEXT),
        Property(name="article", data_type=DataType.TEXT),
        Property(name="url", data_type=DataType.TEXT),
    ],
)

## Load financial articles data

Import the financial news dataset using batch processing for efficient loading.

In [3]:
import pandas as pd
from tqdm import tqdm
from weaviate.util import generate_uuid5

# Load the financial articles dataset
df = pd.read_parquet("data/fin_news_articles_5000.parquet")
df = df[:100]

print(f"Loaded {len(df)} articles")
print("Sample data:")
df.head()

Loaded 100 articles
Sample data:


,date,article_title,stock_symbol,url,article
0,2022-08-04 00:00:00 UTC,Ansys Q2 22 Earnings Conference Call At 8:30 A...,ANSS,https://www.nasdaq.com/articles/ansys-q2-22-ea...,(RTTNews) - Ansys Inc. (ANSS) will host a conf...
1,2012-11-26 00:00:00 UTC,Corrections Corporation of America (CXW) Ex-Di...,PAYX,https://www.nasdaq.com/articles/corrections-co...,Corrections Corporation of America ( CXW ) has...
2,2023-07-14 00:00:00 UTC,"Technology Sector Update for 07/14/2023: ASML,...",ASML,https://www.nasdaq.com/articles/technology-sec...,"Tech stocks were lower late Friday, with the T..."
3,2020-02-16 00:00:00 UTC,3 Best Biotech Stocks to Buy for the Next Decade,VRTX,https://www.nasdaq.com/articles/3-best-biotech...,Let me start out by acknowledging that predict...
4,2021-08-24 00:00:00 UTC,iShares U.S. Transportation ETF Experiences Bi...,CSX,https://www.nasdaq.com/articles/ishares-u.s.-t...,Looking today at week-over-week shares outstan...


In [ ]:
# Get collection reference
articles = client.collections.use("FinancialArticles")

# Import data with batch processing
with articles.batch.fixed_size(batch_size=100, concurrent_requests=2) as batch:
    for _, row in tqdm(df.iterrows(), total=len(df)):
        # Create object from dataframe row
        obj = {
            "article_title": row["article_title"],
            "article": row["article"],
            "url": row["url"] if "url" in row else ""
        }

        # Generate UUID to prevent duplicates
        uuid = generate_uuid5(row["article_title"] + str(row.get("url", "")))

        batch.add_object(
            properties=obj,
            uuid=uuid
        )

        # Check for errors during import
        if batch.number_errors > 10:
            print("Too many errors during import")
            break

print(f"\nImported {len(articles)} articles")

100%|██████████| 100/100 [00:00<00:00, 19349.99it/s]



Imported 100 articles


## Check for import errors

In [5]:
if len(articles.batch.failed_objects) > 0:
    print(f"Import completed with {len(articles.batch.failed_objects)} errors")
    for err in articles.batch.failed_objects[:5]:  # Show first 5 errors
        print(err)
else:
    print("Import completed successfully with no errors")

Import completed successfully with no errors


## Verify the data

Check that articles were imported correctly with embeddings.

In [6]:
# Check collection size
print(f"Total articles in collection: {len(articles)}")

# Show sample data
response = articles.query.fetch_objects(limit=3)

for item in response.objects:
    print(f"Title: {item.properties['article_title']}")
    print(f"Content preview: {item.properties['article'][:100]}...\n")

Total articles in collection: 100
Title: Why Twenty-First Century Fox Stock Jumped 39% Last Year
Content preview: What happened
Shares of Twenty-First Century Fox (NASDAQ: FOXA) surged last year as the entertainmen...

Title: Accenture forecasts fourth-quarter revenue below estimates on forex hit
Content preview: adds forecast, recasts lead, shares
June 23 (Reuters) - IT services company Accenture Plc ACN.N fore...

Title: Noteworthy Tuesday Option Activity: GILD, CMI, AR
Content preview: Looking at options trading activity among components of the Russell 3000 index, there is noteworthy ...



In [7]:
# Verify embeddings were created
response = articles.query.fetch_objects(
    limit=2,
    include_vector=["title", "content"]
)

for item in response.objects:
    print(f"Article: {item.properties['article_title']}")
    print(f"Title vector dimension: {len(item.vector['title'])}")
    print(f"Content vector dimension: {len(item.vector['content'])}\n")

Article: Why Twenty-First Century Fox Stock Jumped 39% Last Year
Title vector dimension: 1024
Content vector dimension: 1024

Article: Accenture forecasts fourth-quarter revenue below estimates on forex hit
Title vector dimension: 1024
Content vector dimension: 1024



## Test basic search

Quick test to verify the collection works with semantic search.

In [8]:
# Test semantic search on titles
response = articles.query.near_text(
    query="technology earnings",
    target_vector="title",
    limit=3
)

print("Search results for 'technology earnings':")
for item in response.objects:
    print(f"- {item.properties['article_title']}")

Search results for 'technology earnings':


## Close the client

Always close your connection when finished.

In [9]:
client.close()